<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys
import os

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.reader import get_all_trajectory_files, read_trajectory_file

from tqdm import tqdm_notebook, tqdm
from ipywidgets import *

%load_ext autoreload
%autoreload 1
%aimport preprocessing, plotting
%matplotlib notebook